<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Generate leads from post likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #post #comments #naas_drivers #automation

**Author:** [Alok Chilka](https://www.linkedin.com/in/calok64/)

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json

### Setup your LinkedIn
- Get your cookies : <a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# Lindekin cookies
LI_AT = "AQEDARCNSioFAHAEAAABfvQsStIAAAF_bznLB1YAQ-SrszGZkuO0fJmeOovASSJfeh5BPTa8J6in6IEnGjeK8IkLVQ-0WdAA41XKFYEb6db3UIGfCPkr9WpirSF4jwB0qKMYT0mHvDIpgwNcZJV20lob"
JSESSIONID = "ajax:2913276942853423494"

# Enter your profile URL
PROFILE_URL = "https://www.linkedin.com/in/florent-ravenel/"

### Setup your HubSpot
👉 Access your [HubSpot API key](https://knowledge.hubspot.com/integrations/how-do-i-get-my-hubspot-api-key)<br>
👉 Get your HubSpot owner ID<br>
👉 Get your HubSpot company ID from URL

In [ ]:
# HubSpot API key
HS_API_TOKEN = "YOUR_HUBSPOT_API_KEY"

# Your company located after "reports-dashboard/" when you are connected in HubSpot https://app.hubspot.com/reports-dashboard/2474088/view/244930
HS_COMPANY_ID = "2474088"

# Contact owner => contact id to whom task needs to be assigned
HS_OWNER_ID = "41252017"

# Time delay to set due date for tasks in days
time_delay = 10

### Setup your email to receive notifications

In [ ]:
SEND_EMAIL_TO = ""

## Model

### Get posts from LinkedIn feed

In [ ]:
df_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(PROFILE_URL, count=100, limit=-1)
df_posts

### Get likes from LinkedIn posts

In [ ]:
def get_likes(df):
    df_likes = pd.DataFrame()
    for index, row in df.iterrows():
        post_url = row['POST_URL']
        tmp_df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        if not tmp_df.empty:
            df_likes = pd.concat([df_likes, tmp_df])
    return df_likes
        
df_likes = get_likes(df_posts)
df_likes

### Get number of likes by profile

In [ ]:
def count_likes(df):
    to_group = ["PROFILE_URN",
                "PROFILE_ID",
                "FIRSTNAME",
                "LASTNAME"]
    df = df.groupby(to_group, as_index=False).agg({"POST_URL": "count"})
    df = df.sort_values(by="POST_URL", ascending=False)
    return df 

df_counts = count_likes(df_likes)
df_counts

### Apply classification to get potential lead

- Grouping the like count and classify the leads as per levels i.e. <b>L1, L2, L3, L4.</b> ( from highest to lowest counts of likes) with column name <b>"POTENTIAL LEAD"</b>
- Group posts as per the the column <b>"POTENTIAL LEAD"</b>

In [ ]:
max_likes = df_counts.POST_URL.max()
cluster_1 = round(max_likes * 0.1, 0)
cluster_2 = round(max_likes * 0.5, 0)
cluster_3 = round(max_likes * 0.8, 0)

In [ ]:
conditions = [
    (df_counts['POST_URL'] <= cluster_1),
    (df_counts['POST_URL'] > cluster_1) & (df_counts['POST_URL'] <= cluster_2),
    (df_counts['POST_URL'] > cluster_2) & (df_counts['POST_URL'] <= cluster_3),
    (df_counts['POST_URL'] > cluster_3)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [ ]:
df_counts['POTENTIAL_LEAD'] = np.select(conditions, values)
df_counts

### Get profile details based on the POST_URL above
- filter rows from "grouped_profile_posts" dataframe based on L1, L2, & L3 levels and add data to leads_list
- Extract EMAIL, FIRSTNAME, LASTNAME, PHONE_NUMBER, OCCUPATION from profile URL

In [ ]:
leads_list = [];

r_count = 1
for index, row in df_counts.iterrows():
    if row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_URN']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
       
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilephoneno = PROFILECONTACTS.at[0,'PHONENUMBER']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        profilefirstname = PROFILEIDENTITY.at[0,'FIRSTNAME']
        profilelastname = PROFILEIDENTITY.at[0,'LASTNAME']
        profileoccupation = PROFILEIDENTITY.at[0,'OCCUPATION']
        if profileemail != None :
            leads_list.append([r_count, profilename, profilefirstname, profilelastname, profileemail, profilephoneno, profileoccupation, profileurl])
            r_count = r_count + 1

In [ ]:
pd.DataFrame(leads_list)

### Create hubspot contacts from linkedin likes

<h4>HS_CONTACT_ID_LIST</h4> - The list to store the contact id recently created and the linkedin profile URL of associated contact

In [ ]:
#to store the resulting output of create contact method
huboutput = ""

HS_CONTACT_ID_LIST = []
for i in leads_list:
        profilename = i[1]
        profilefirstname = i[2]
        profilelastname = i[3]
        profileemail = i[4]
        profilephoneno = i[5]
        profileoccupation = i[6]
        profileurl = i[7]
        # With send method
        data = {"properties": 
                {
                    "linkedinbio": profileurl,
                    "firstname": profilefirstname,
                    "lastname": profilelastname,
                    "jobtitle": profileoccupation,
                    "email": profileemail,
                    "phone": profilephoneno,
                }
               }
        
        #write data to CRM ( create new contact in CRM)
        huboutput = hubspot.connect(HS_API_TOKEN).contacts.send(data)
        HS_CONTACT_ID_LIST.append([huboutput, profileurl])

## Output

### Create tasks for user and send email notification

In [ ]:
def create_task(owner_id,
                time_delay,
                subject,
                body,
                status,
                asso_contactids=[],
                asso_companyids=[],
                asso_dealids=[],
                asso_ownerids=[],
                engagement="TASK"):
    """
    Engagement type = TASK | NOTE | EMAIL | MEETING | CALL 
    """
    
    # Calc timestamp
    tstampobj = datetime.now() + timedelta(days=time_delay)
    tstamp = tstampobj.timestamp() * 1000
     
    # Create payload
    payload = json.dumps({
        "engagement": {
            "active": 'true',
            "ownerId": owner_id,
            "type": engagement,
            "timestamp": timestamp
        },
        "associations": {
            "contactIds": asso_contactids,
            "companyIds": asso_companyids,
            "dealIds": asso_dealids,
            "ownerIds": asso_ownerids,
        },
        "metadata": {
            "body": body,
            "subject": subject,
            "status": status,
        }
    })
    url = "https://api.hubapi.com/engagements/v1/engagements"
    params = {"hapikey": HS_API_TOKEN}
    headers = {'Content-Type': "application/json"}
    # Post requests
    res = requests.post(url,
                        data=payload,
                        headers=headers,
                        params=params)
    # Check requests
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        raise (e)
    res_json = res.json()
    
    # Fetch the task id of the current task created
    task_id = res_json.get("engagement").get("id")
    print("🎆 Task created successfully: ", task_id)
    return res_json

In [ ]:
HS_CONTACT_URL = "https://app.hubspot.com/contacts/"+HS_COMPANY_ID+"/contact/"

for i in HS_CONTACT_ID_LIST:
    contact_id = i[0]
    contact_linkedin_url = i[1]
    hs_contact_url = HUB_CONTACT_URL + contact_id
    contact = hubspot.connect(HS_API_TOKEN).get(contact_id)
    contact_props = contact.get('properties')
    
    subject = "Task created for Contact ID :"+ contact_id,
    status = "NOT_STARTED"
    body = "Hi there, you need to contact following user & task is already assigned to you.<br/>" + "Name :" +contact_props['firstname']+ contact_props['lastname'] + " Contact URL : " + hs_contact_url,
    
    # Create task in HubSpot
    try:
        create_task(owner_id=HS_OWNER_ID,
                    time_delay=time_delay
                    subject=subject,
                    body=body,
                    status=status,
                    asso_contactids=[contact_id],
                    engagement="TASK")

        print("Task created successfully for Contact ID :", contact_id)
        email_to = SEND_EMAIL_TO #to send the report
        subject = "LinkedIN Leads Alert"
        content = f"<p>Hi there,</p><br/><p>A new task has been created for you with contact ID :" + contact_id + "</p><br/><p>Actions : <a href="+hs_contact_url+">Open Hubspot Contact</a> | <a href="+contact_linkedin_url+">Open LinkedIN Profile</a></p>"
        naas.notification.send(email_to=email_to,
                               subject=subject,
                               html=content)
    except:
        print("Could not create the task for Contact ID :"+contact_id+", ask for on hello@naas.ai for more details")